In [179]:
import numpy as np
from matplotlib import pyplot as plt

# reference distance (meters)
d0 = 1 
# constant for particular propagation enviroment
k = 1e-4
# path loss exponent
n = 4
# total available bandwith (Hz)
b_t = 100 * 1e6
# constant for noise power (W/Hz)
k_0 = 1e-20

def lin2db(x):
    return 10.0*np.log10(x)

def db2lin(x):
    return 10.0**(x/10.0)

def lin2dbm(x):
    return 10.0*np.log10(x)+30.0

def dbm2lin(x):
    return 10.0**(x/10.0 - 3.0)

def bandwith4channel(total_bandwith, channels_quantity):
    return total_bandwith / channels_quantity

def received_power(transmitted_power, distance):
    return transmitted_power * (k / (distance**n))

def noise_power(total_bandwith, channels_quantity):
    return k_0 * bandwith4channel(total_bandwith, channels_quantity)

In [180]:
# Models the UEs
class UserEquipment:
    def __init__(self, power_ = 1): # UE transmission power is 1 W by standard
        # Distance is sorted randomly in a 1000x1000m area 
        self.distance2AP = np.sqrt((np.random.rand()*1e3 - 500)**2 
                                    + (np.random.rand()*1e3 - 500)**2) 
        
        self.power = power_

In [181]:
# Creates and stores the UES
class UserGenerator:
    def __init__(self, quantity_):
        self.quantity = quantity_
        
        # list of powers receiveds by AP (watts)
        self.users_power = np.zeros(quantity_)
        
        for i in range(quantity_):
            temp_user = UserEquipment()
            if temp_user.distance2AP >= d0:
                self.users_power[i] = received_power(temp_user.power, 
                                                     temp_user.distance2AP)
                
        # Allocating channels
        self.channels = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}
        
        aux_channel = 0
        for value in range(len(self.users_power)):
            if aux_channel <= len(self.channels) - 1:
                self.channels[aux_channel].append(self.users_power[value])
                aux_channel += 1
            else:
                aux_channel = 1
                self.channels[aux_channel].append(self.users_power[value])
                
        print(self.channels)

In [182]:
# Admiting only 1 channel, the noise power
noise_p = noise_power(b_t, 10)

# Models a System to manage the link between UE and AP
class LinkSystem:
    def __init__(self, users_generator_):
        if isinstance(users_generator_, UserGenerator):
            self.SNR_list = np.zeros(len(users_generator_.users_power))
            self.SINR_list = np.zeros(len(users_generator_.users_power))
            self.capacity_list = np.zeros(len(self.SINR_list))
            
            # Calculating SNR
            for user_ in range(len(users_generator_.users_power)):
                self.SNR_list[user_] = lin2db(users_generator_.users_power[user_]) / lin2db(noise_p)
            
            for ch in range(len(users_generator_.channels)):
                # Calculating SINR
                for user__ in range(len(users_generator_.channels[ch])):
                    auxiliar_received_power = list(users_generator_.channels[ch])
                    print(user__)
                    UE_used = users_generator_.channels[ch][user__]
                    users_generator_.channels[ch].pop(user__)
                    interfering_sum = sum(auxiliar_received_power)
                    self.SINR_list[user__] = lin2db(UE_used) / (lin2db(interfering_sum) + lin2db(noise_p))

            print(self.SINR_list) 
                
                # Calculating Capacity
                # for user_SINR_ in range(len(self.SINR_list)):
                #    bandwith_ = bandwith4channel(b_t, 10)
                #    self.capacity_list[user_SINR_] = (bandwith_ * np.log2(1 + db2lin(self.SINR_list[user_SINR_]))) / 1e6

In [183]:
# Create a UE using UserEquipment class
UE1 = UserEquipment()
print(UE1.distance2AP, UE1.power)

iteracoes = 1
total_SNR = []
total_SINR = []
total_capacity = []

for inter in range(iteracoes):
    # Creates many Users at once
    users = UserGenerator(20)
    
    # The SNR and SINR of users
    system = LinkSystem(users)
    
    total_SNR = np.concatenate((total_SNR, system.SNR_list))
    
    total_SINR = np.concatenate((total_SINR, system.SINR_list))
    
    total_capacity = np.concatenate((total_capacity, system.capacity_list))
    
print(np.sort(total_capacity))

400.0985473710563 1
{0: [3.4774319718210178e-15], 1: [7.628961866655426e-16, 1.0761639254996356e-14, 2.175530560267162e-15], 2: [1.2190161508033646e-14, 4.528891232005756e-15], 3: [1.6695962165767904e-14, 1.2504742397181438e-13], 4: [2.7682534793588362e-14, 1.0339675047872456e-13], 5: [1.085130729918292e-14, 9.836445371681153e-16], 6: [8.00998803451391e-15, 8.389578797833887e-14], 7: [1.7193221795684427e-14, 6.2808117092849355e-15], 8: [9.208728554159088e-16, 1.283113360221008e-12], 9: [3.5841844005427424e-15, 1.358547740104188e-15]}
0
0
1
2


IndexError: list index out of range

In [ ]:
x = np.sort(total_capacity)
y = np.arange(0, len(total_capacity)) / len(total_capacity)

plt.xlabel('capacity (Mbps)')
plt.ylabel('CDF')

plt.plot(x, y)
plt.show()